# Dia 1 - Importação de Dados

In [ ]:
# Importando bibliotecas

# !pip install pyarrow
# !pip install pandas

import pandas as pd
import glob

### Criando repositórios para os arquivos

In [3]:
# Criando diretórios e pastas para receber os arquivos do GitHub

from pathlib import Path

# Cria um caminho (sem criar ainda no sistema de arquivos)
pasta_emprestimos = Path("dados/emprestimos")
pasta_exemplares = Path("dados/exemplares")

# Criação das pastas, incluindo as intermediárias se não existirem
pasta_emprestimos.mkdir(parents=True, exist_ok=True)
pasta_exemplares.mkdir(parents=True, exist_ok=True)

### Realizando a coleta de dados

In [4]:
# Baixando os arquivos para as pastas correspondentes

import requests

# Baixando CSVs

# Lista de anos para concatenar o nome do arquivo.
# Os arquivos CSV tem o mesmo prefixo mudando apenas o final.
# Para não importar os arquivos um a um, cria-se uma lista com o nome final,
# e concatena com o prefixo, realizando o download com um loop
lista_emprestimos = []

# Criando a lista automáticamente
for ano in range(2010, 2021):
    for semestre in [1, 2]:
        if ano == 2020 and semestre == 2:
            continue # pula o segundo semestre de 2020 e encerra o loop
        lista_emprestimos.append(f'{ano}{semestre}')

# Loop de download
for ano in lista_emprestimos:
    # passando o link para download e concatenando o ano ao final do nome do arquivo
    url = f"https://raw.githubusercontent.com/franciscofoz/7_Days_of_Code_Alura-Python-Pandas/main/Dia_1-Importando_dados/Datasets/dados_emprestimos/emprestimos-{ano}.csv"
    
    # faz uma requisição HTTP e armazena (get) a resposta em r
    r = requests.get(url)
    
    # armazenando o arquivo na pasta correspondente
    with open(f"dados/emprestimos/emprestimos-{ano}.csv", "wb") as f:
        f.write(r.content) # escreve o conteúdo de "r", que é o que ele pegou do link, o arquivo.

# Baixando o Parquet

parquet_url = f"https://github.com/franciscofoz/7_Days_of_Code_Alura-Python-Pandas/raw/main/Dia_1-Importando_dados/Datasets/dados_exemplares.parquet"
r = requests.get(parquet_url)

with open("dados/exemplares/dados_exemplares.parquet", "wb") as f:
    f.write(r.content)

In [5]:
# Criando uma lista com os caminhos de todos os arquivos .csv na pasta emprestimos
caminhos_csv = glob.glob("dados/emprestimos/*.csv")
print(caminhos_csv)

['dados/emprestimos\\emprestimos-20101.csv', 'dados/emprestimos\\emprestimos-20102.csv', 'dados/emprestimos\\emprestimos-20111.csv', 'dados/emprestimos\\emprestimos-20112.csv', 'dados/emprestimos\\emprestimos-20121.csv', 'dados/emprestimos\\emprestimos-20122.csv', 'dados/emprestimos\\emprestimos-20131.csv', 'dados/emprestimos\\emprestimos-20132.csv', 'dados/emprestimos\\emprestimos-20141.csv', 'dados/emprestimos\\emprestimos-20142.csv', 'dados/emprestimos\\emprestimos-20151.csv', 'dados/emprestimos\\emprestimos-20152.csv', 'dados/emprestimos\\emprestimos-20161.csv', 'dados/emprestimos\\emprestimos-20162.csv', 'dados/emprestimos\\emprestimos-20171.csv', 'dados/emprestimos\\emprestimos-20172.csv', 'dados/emprestimos\\emprestimos-20181.csv', 'dados/emprestimos\\emprestimos-20182.csv', 'dados/emprestimos\\emprestimos-20191.csv', 'dados/emprestimos\\emprestimos-20192.csv', 'dados/emprestimos\\emprestimos-20201.csv']


### Criando um DataFrame de DataFrames

In [ ]:
# Adicionando todos os CSVs em um único DataFrame

dataframes = []

for caminho in caminhos_csv: 
    print(f'Lendo o arquivo: {caminho}') 
    try:
        df = pd.read_csv(caminho, sep = ',', encoding = 'utf-8') 
        print(f'Primeiras linhas do arquivo {caminho}:')
        print(df.head()) 
        dataframes.append(df)
    except Exception as e:
        print(f'Erro ao ler {caminho}: {e}')

if not dataframes:
    print(f'Nenhum arquivo foi lido corretamente!')
else:
    df_emprestimos = pd.concat(dataframes, ignore_index=True)
    print("DataFrame final concatenado:")
    print(df_emprestimos.head())

Lendo o arquivo: dados/emprestimos\emprestimos-20101.csv
Erro ao ler dados/emprestimos\emprestimos-20101.csv: No columns to parse from file
Lendo o arquivo: dados/emprestimos\emprestimos-20102.csv
Primeiras linhas do arquivo dados/emprestimos\emprestimos-20102.csv:
   id_emprestimo                codigo_barras                 data_renovacao  \
0         799100  S025602_19/11/2013 10:51:34                            NaN   
1         799101                   2008049500                            NaN   
2         799102                      X008043                            NaN   
3         799103                   2009055438  2010/07/14 18:35:35.042000000   
4         799104                      S024636  2010/07/16 21:29:43.569000000   

                 data_emprestimo                 data_devolucao  \
0  2010/07/01 07:45:41.111000000  2010/07/07 10:52:31.000000000   
1  2010/07/01 08:08:17.690000000  2010/07/01 09:03:44.000000000   
2  2010/07/01 08:13:31.574000000  2010/07/09 15:14:4

### Concatenando os DataFrames

In [7]:
# Juntando todas as tabelas em uma única

df_emprestimos.head()

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario
0,799100,S025602_19/11/2013 10:51:34,NaN,2010/07/01 07:45:41.111000000,2010/07/07 10:52:31.000000000,2.009050e+09,ALUNO DE GRADUAÇÃO
1,799101,2008049500,NaN,2010/07/01 08:08:17.690000000,2010/07/01 09:03:44.000000000,2.010056e+09,ALUNO DE GRADUAÇÃO
2,799102,X008043,NaN,2010/07/01 08:13:31.574000000,2010/07/09 15:14:44.000000000,2.009043e+09,ALUNO DE GRADUAÇÃO
3,799103,2009055438,2010/07/14 18:35:35.042000000,2010/07/01 08:22:46.085000000,2010/07/29 14:59:16.000000000,2.009054e+09,ALUNO DE GRADUAÇÃO
4,799104,S024636,2010/07/16 21:29:43.569000000,2010/07/01 08:40:28.512000000,2010/07/28 07:49:32.000000000,2.009054e+09,ALUNO DE GRADUAÇÃO


In [8]:
df_emprestimos.columns

Index(['id_emprestimo', 'codigo_barras', 'data_renovacao', 'data_emprestimo',
       'data_devolucao', 'matricula_ou_siape', 'tipo_vinculo_usuario'],
      dtype='object')

### Verificando valores nulos

In [9]:
# Verificando e tratando valores nulos

df_emprestimos.isnull().sum()

id_emprestimo                 0
codigo_barras                 0
data_renovacao          1167121
data_emprestimo               0
data_devolucao             6443
matricula_ou_siape         2799
tipo_vinculo_usuario          0
dtype: int64

### Verificando Duplicatas

In [10]:
# Verificando duplicatas no DataFrame inteiro

df_emprestimos.duplicated().sum()

np.int64(19)

In [34]:
df_emprestimos.value_counts()

id_emprestimo  codigo_barras  data_renovacao           data_emprestimo          data_devolucao       matricula_ou_siape  tipo_vinculo_usuario  
799103         2009055438     2020-01-17 09:56:18.380  2020-01-02 08:36:40.163  2020-01-31 14:53:52  2.009054e+09        ALUNO DE GRADUAÇÃO        1
811090         L168864        2020-03-07 20:07:35.426  2020-02-21 15:01:53.538  2021-04-08 15:11:41  2.009052e+09        ALUNO DE GRADUAÇÃO        1
811115         L148121        2020-03-09 08:53:34.303  2020-02-21 15:15:10.526  2020-11-11 16:47:47  2.010058e+09        ALUNO DE GRADUAÇÃO        1
811114         2008032900     2020-03-09 08:53:34.329  2020-02-21 15:15:10.470  2020-11-11 16:47:27  2.010034e+09        ALUNO DE GRADUAÇÃO        1
811113         L178405        2020-03-05 05:39:39.100  2020-02-21 15:14:01.944  2020-03-06 12:01:08  2.010049e+09        ALUNO DE GRADUAÇÃO        1
                                                                                                               

In [11]:
# Mostra todas as linhas duplicadas

df_emprestimos[df_emprestimos.duplicated()]

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario
1299549,2351375,L057436,2016/01/31 07:30:36.293000000,2016/01/02 08:38:40.009000000,2016/02/21 13:16:29.000000000,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO
1299550,2351376,H004983,2016/01/31 07:31:36.786000000,2016/01/02 08:46:42.991000000,2016/02/22 16:10:28.000000000,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO
1299551,2351377,H011333,2016/01/31 07:31:36.658000000,2016/01/02 08:46:43.400000000,2016/02/22 16:10:40.000000000,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO
1299567,2351385,2013114886,2016/01/16 07:41:19.590000000,2016/01/02 10:02:44.456000000,2016/01/30 07:52:10.000000000,2.015316e+09,ALUNO MÉDIO/TÉCNICO
1299568,2351386,2015005192,2016/01/16 07:41:19.537000000,2016/01/02 10:02:44.507000000,2016/01/30 07:52:07.000000000,2.015316e+09,ALUNO MÉDIO/TÉCNICO
1299569,2351387,2015004940,2016/01/16 07:41:19.473000000,2016/01/02 10:02:44.560000000,2016/01/30 07:52:00.000000000,2.015316e+09,ALUNO MÉDIO/TÉCNICO
1299570,2351388,2012029817,NaN,2016/01/02 10:08:27.599000000,2016/01/23 16:28:01.000000000,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO
1299571,2351389,2014077426,NaN,2016/01/02 10:08:27.658000000,2016/01/23 16:27:57.000000000,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO
1299572,2351390,2015007050,2016/01/31 23:03:17.389000000,2016/01/02 10:08:27.720000000,2016/03/16 16:26:37.000000000,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO
1299573,2351391,L195470,2016/01/17 16:03:38.760000000,2016/01/02 10:12:01.462000000,2016/01/27 16:37:55.000000000,2.014036e+09,ALUNO DE GRADUAÇÃO


In [35]:
# Garantindo que as linhas são realmente duplicadas

df_emprestimos[df_emprestimos.duplicated(keep=False)]

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario
1299546,2351375,L057436,NaT,NaT,NaT,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO
1299547,2351376,H004983,NaT,NaT,NaT,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO
1299548,2351377,H011333,NaT,NaT,NaT,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO
1299549,2351375,L057436,NaT,NaT,NaT,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO
1299550,2351376,H004983,NaT,NaT,NaT,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO
1299551,2351377,H011333,NaT,NaT,NaT,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO
1299559,2351385,2013114886,NaT,NaT,NaT,2.015316e+09,ALUNO MÉDIO/TÉCNICO
1299560,2351386,2015005192,NaT,NaT,NaT,2.015316e+09,ALUNO MÉDIO/TÉCNICO
1299561,2351387,2015004940,NaT,NaT,NaT,2.015316e+09,ALUNO MÉDIO/TÉCNICO
1299562,2351388,2012029817,NaT,NaT,NaT,2.016101e+10,ALUNO DE PÓS-GRADUAÇÃO


### Excluindo duplicatas

In [37]:
df_emprestimos = df_emprestimos.drop_duplicates()
df_emprestimos.value_counts()

id_emprestimo  codigo_barras  data_renovacao           data_emprestimo          data_devolucao       matricula_ou_siape  tipo_vinculo_usuario  
799103         2009055438     2020-01-17 09:56:18.380  2020-01-02 08:36:40.163  2020-01-31 14:53:52  2.009054e+09        ALUNO DE GRADUAÇÃO        1
811090         L168864        2020-03-07 20:07:35.426  2020-02-21 15:01:53.538  2021-04-08 15:11:41  2.009052e+09        ALUNO DE GRADUAÇÃO        1
811115         L148121        2020-03-09 08:53:34.303  2020-02-21 15:15:10.526  2020-11-11 16:47:47  2.010058e+09        ALUNO DE GRADUAÇÃO        1
811114         2008032900     2020-03-09 08:53:34.329  2020-02-21 15:15:10.470  2020-11-11 16:47:27  2.010034e+09        ALUNO DE GRADUAÇÃO        1
811113         L178405        2020-03-05 05:39:39.100  2020-02-21 15:14:01.944  2020-03-06 12:01:08  2.010049e+09        ALUNO DE GRADUAÇÃO        1
                                                                                                               

In [38]:
# Verificando duplicatas no DataFrame inteiro

df_emprestimos.duplicated().sum()

np.int64(0)

In [39]:
# Mostra todas as linhas duplicadas

df_emprestimos[df_emprestimos.duplicated()]

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario


### Últimos ajustes no DataFrame de empréstimos

In [40]:
# Verificando os tipos de dados do DataFrame

df_emprestimos.dtypes

id_emprestimo                    int64
codigo_barras                   object
data_renovacao          datetime64[ns]
data_emprestimo         datetime64[ns]
data_devolucao          datetime64[ns]
matricula_ou_siape             float64
tipo_vinculo_usuario            object
dtype: object

In [41]:
# Verificando valores nulos na coluna código de barras, que será a relação com o arquivo dados do acervo

df_emprestimos['codigo_barras'].isnull().sum()

np.int64(0)

In [42]:
# Verificando padrão das colunas de datas

df_emprestimos['data_devolucao'] = pd.to_datetime(df['data_devolucao'], errors = 'coerce')
df_emprestimos['data_emprestimo'] = pd.to_datetime(df['data_emprestimo'], errors = 'coerce')
df_emprestimos['data_renovacao'] = pd.to_datetime(df['data_renovacao'], errors = 'coerce')

### Importando os dados do acervo

In [43]:
# Carregando o arquivo de dados do acervo

df_acervo = pd.read_parquet('dados/exemplares/dados_exemplares.parquet')
df_acervo.head()

,id_exemplar,codigo_barras,colecao,biblioteca,status_material,localizacao,registro_sistema
index,,,,,,,
0,5,L000003,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,694,1
1,4,L000002,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,688,1
2,3,L000001,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,638,1
3,7,L000114,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,616,5
5,10,L000041,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,657,15


### Validações do DataFrame de acervo

In [44]:
# Verificações do arquivo de dados do acervo

df_acervo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 546237 entries, 0 to 568584
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id_exemplar       546237 non-null  int64 
 1   codigo_barras     546237 non-null  object
 2   colecao           546237 non-null  object
 3   biblioteca        546237 non-null  object
 4   status_material   546237 non-null  object
 5   localizacao       546237 non-null  int64 
 6   registro_sistema  546237 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 33.3+ MB


In [45]:
# Verifica valores nulos

df_acervo.isnull().sum()

id_exemplar         0
codigo_barras       0
colecao             0
biblioteca          0
status_material     0
localizacao         0
registro_sistema    0
dtype: int64

In [46]:
# verifica colunas únicas ou  irrelevantes

df_acervo.nunique()

id_exemplar         546141
codigo_barras       545049
colecao                 18
biblioteca              25
status_material          3
localizacao            900
registro_sistema    198503
dtype: int64

In [47]:
# Visualiza amostras aleatórias

df_acervo.sample(5)

,id_exemplar,codigo_barras,colecao,biblioteca,status_material,localizacao,registro_sistema
index,,,,,,,
315495,809297,2010063456,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,611,134370
107827,120420,L036747,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,699,39650
252344,252471,2008041782,Acervo Circulante,Biblioteca Setorial Prof. Francisco Gurgel De ...,REGULAR,610,107083
271148,821093,2010064886,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,623,115955
505065,1216069,2016003516,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,590,228796


In [48]:
# Verifica o tipo de dados

df_acervo.dtypes

id_exemplar          int64
codigo_barras       object
colecao             object
biblioteca          object
status_material     object
localizacao          int64
registro_sistema     int64
dtype: object

### Unindo os dois DataFrames

In [49]:
# Mesclando os DataFrames de Emprestimos e Acervo

# O método merge, irá mesclar os DataFrames, usando como referência a coluna 'codigo_barras'
df_completo = pd.merge(df_emprestimos, # DataFrame da esquerda
                       df_acervo, # DataFrame da direita
                       on = 'codigo_barras', # Coluna em comum entre os dois
                       how = 'left' # Tipo de junção (LEFT JOIN)
                       )
# how = left significa: "traga todos os registros do DataFrame da esquerda (df_emprestimos) e complemente com os dados do da direita (df_acervo) quando houver correspondência"

df_completo.head()

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,registro_sistema
0,799100,S025602_19/11/2013 10:51:34,NaT,2020-01-02 08:17:30.290,2020-01-07 11:14:07,2.009050e+09,ALUNO DE GRADUAÇÃO,186525.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,635.0,71668.0
1,799101,2008049500,NaT,2020-01-02 08:17:30.391,2020-01-07 11:14:17,2.010056e+09,ALUNO DE GRADUAÇÃO,184388.0,Acervo Circulante,Biblioteca Setorial Prof. Francisco Gurgel De ...,REGULAR,694.0,70925.0
2,799102,X008043,NaT,2020-01-02 08:17:30.436,2020-01-07 11:24:46,2.009043e+09,ALUNO DE GRADUAÇÃO,161925.0,Acervo Circulante,Biblioteca Setorial Prof. Ronaldo Xavier de Ar...,REGULAR,613.0,60177.0
3,799103,2009055438,2020-01-17 09:56:18.380,2020-01-02 08:36:40.163,2020-01-31 14:53:52,2.009054e+09,ALUNO DE GRADUAÇÃO,286093.0,Acervo Circulante,Biblioteca Setorial do Centro Ciências da Saúd...,REGULAR,690.0,125251.0
4,799104,S024636,2020-01-17 23:50:18.862,2020-01-02 09:36:03.877,2020-01-24 08:18:19,2.009054e+09,ALUNO DE GRADUAÇÃO,158723.0,Acervo Circulante,Biblioteca Setorial do Centro Ciências da Saúd...,REGULAR,637.0,58978.0


### Verificando se há linhas sem correspondência e o percentual

In [50]:
# Verificando quantas linhas não encontraram correspondência no acervo

faltando_acervo = df_completo['id_exemplar'].isnull().sum()
print(f'Linhas sem correspondência no acervo: {faltando_acervo}')

percent_faltando = (faltando_acervo / len(df_completo)) * 100
print(f'{percent_faltando:.2f}% dos empréstimos não têm dados de acervo.')

Linhas sem correspondência no acervo: 168727
8.19% dos empréstimos não têm dados de acervo.


### Verificando quais os códigos de barra que não encontraram correspondência

In [51]:
# Verificando quais códigos de barras não encontraram correspondência

# Seleciona a coluna 'id_exemplar' que vem de df_acervo. Se ela tiver NaN após o merge, não houve correspondência para 
# Aquele código de barras.
# isna() retorna True para valores NaN, e False para valores preenchidos
# df_sem_match está recebendo os emprestimos que não tiveram correspondência com os ID's de exemplares do acervo.
df_sem_match = df_completo[df_completo['id_exemplar'].isna()]
df_sem_match.head()

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,registro_sistema
12,799112,X046-17,2020-02-01 09:32:51.569,2020-01-02 09:54:30.184,2020-07-27 11:38:04,2.009116e+09,ALUNO DE PÓS-GRADUAÇÃO,NaN,NaN,NaN,NaN,NaN,NaN
13,799113,X046-20,2020-02-01 09:32:51.543,2020-01-02 09:54:30.220,2020-07-27 11:38:28,2.009116e+09,ALUNO DE PÓS-GRADUAÇÃO,NaN,NaN,NaN,NaN,NaN,NaN
14,799114,X046-11,NaT,2020-01-02 10:08:33.890,2020-02-03 17:11:45,2.009116e+09,ALUNO DE PÓS-GRADUAÇÃO,NaN,NaN,NaN,NaN,NaN,NaN
15,799115,X046-12,NaT,2020-01-02 10:08:33.930,2020-02-03 17:11:34,2.009116e+09,ALUNO DE PÓS-GRADUAÇÃO,NaN,NaN,NaN,NaN,NaN,NaN
19,799119,X0046-1,2020-02-01 07:51:12.857,2020-01-02 10:24:17.558,2020-03-05 08:18:39,2.009116e+09,ALUNO DE PÓS-GRADUAÇÃO,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
# Retorna os valores únicos da coluna código de barras
# Quais foram os códigos de barras que não foram encontrados no acervo?
df_sem_match['codigo_barras'].unique()

array(['X046-17', 'X046-20', 'X046-11', ..., 'L143612', '2018007270',
       'Y021773'], shape=(36301,), dtype=object)